# Advanced Querying Mongo

Importing libraries and setting up connection

In [17]:
!pip3 install pymongo

In [33]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL)
print(client)
db = client.get_database()
print(db)
cols = db.list_collections()
print(cols)
for x in cols:
    print(x)

mongodb://localhost/companies
MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')
{'name': 'companies', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': UUID('66f0fe67-0302-4902-95b5-aabc2b0ca759')}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_', 'ns': 'companies.companies'}}


In [45]:
client.list_database_names()
db.list_collection_names()
collection = db['companies']
collection 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [70]:
# Your Code
company = list(db.companies.find({"name":'Babelgum'}, {"name":1, "_id":0}))
print(company)

[{'name': 'Babelgum'}]


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [62]:
# Your Code
query = {"number_of_employees": {"$gt": 5000}}
employees = (list(db.companies.find(query, {"name":1, "number_of_employees":1}).sort([("number_of_employees", DESCENDING)]).limit(20)))
for element in employees:
    print(element["name"], element["number_of_employees"])

Siemens 405000
IBM 388000
Toyota 320000
PayPal 300000
Nippon Telegraph and Telephone Corporation 227000
Samsung Electronics 221726
Accenture 205000
Tata Consultancy Services 200300
Flextronics International 200000
Safeway 186000
Sony 180500
LG 177000
Ford 171000
Boeing 160000
Digital Equipment Corporation 140000
Nokia 125000
MItsubishi Electric 107000
MItsubishi Electric 107000
Comcast 100000
Bertelsmann 100000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [116]:
# Your Code
query2 = {"$and":[{"founded_year":{"$gt":2000}},
                 {"founded_year": {"$lt":2005}}
                ]
}
c_years = (list(db.companies.find(query2, {"name":1, "founded_year":1})))
print(len(c_years))
#for element in c_years:
    #print(f'Name of the company: {element["name"]}, founded: {element["founded_year"]}')

2252


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [103]:
# Your Code
query3 = {"$and":[{"ipo.valuation_amount": {"$gt":100000000}}, {"founded_year": {"$lt":2010}}]}
valuation = (list(db.companies.find(query3, {"name":1, "ipo.valuation_amount":1})))
for element in valuation:
    print(f'Name of the company: {element["name"]}, ipo: {element["ipo"]}')


Name of the company: Twitter, ipo: {'valuation_amount': 18100000000}
Name of the company: Facebook, ipo: {'valuation_amount': 104000000000}
Name of the company: Yelp, ipo: {'valuation_amount': 1300000000}
Name of the company: LinkedIn, ipo: {'valuation_amount': 9310000000}
Name of the company: Brightcove, ipo: {'valuation_amount': 290000000}
Name of the company: Amazon, ipo: {'valuation_amount': 100000000000}
Name of the company: KIT digital, ipo: {'valuation_amount': 235000000}
Name of the company: Nielsen, ipo: {'valuation_amount': 1600000000}
Name of the company: OpenTable, ipo: {'valuation_amount': 1050000000}
Name of the company: ChannelAdvisor, ipo: {'valuation_amount': 287000000}
Name of the company: Jive Software, ipo: {'valuation_amount': 1000000000}
Name of the company: Zillow, ipo: {'valuation_amount': 2550000000}
Name of the company: Wix, ipo: {'valuation_amount': 750000000}
Name of the company: Shutterfly, ipo: {'valuation_amount': 350000000}
Name of the company: TripAdvis

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [105]:
# Your Code
query4 = {"$and":[{"number_of_employees": {"$lt":1000}}, {"founded_year": {"$lt":2005}}]}
results = (list(db.companies.find(query4, {"name":1}).sort([("number_of_employees", ASCENDING)]).limit(10)))
for element in results:
    print(f'Name of the company: {element["name"]}')

Name of the company: Fox Interactive Media
Name of the company: Skype
Name of the company: Ticketmaster
Name of the company: stylediary
Name of the company: MindTouch
Name of the company: Simpy
Name of the company: Eurekster
Name of the company: Compete
Name of the company: EditGrid
Name of the company: Monster


In [106]:
query4 = {"$and":[{"number_of_employees": {"$lt":1000}}, {"founded_year": {"$lt":2005}}]}
results = (list(db.companies.find(query4, {"name":1}).sort([("number_of_employees", DESCENDING)]).limit(10)))
for element in results:
    print(f'Name of the company: {element["name"]}')

Name of the company: Infinera Corporation
Name of the company: NorthPoint Communications Group
Name of the company: 888 Holdings
Name of the company: Forrester Research
Name of the company: SonicWALL
Name of the company: Webmetrics
Name of the company: Cornerstone OnDemand
Name of the company: Yelp
Name of the company: ZoomInfo
Name of the company: MySpace


### 6. All the companies that don't include the `partners` field.

In [111]:
# Your Code
query5 = {"partners":{"$exists":False}}
partners = (list(db.companies.find(query5, {"name":1})))
for element in result:
    print(f'Name of the company: {element["name"]}')
print(len(partners))

0


### 7. All the companies that have a null type of value on the `category_code` field.

In [117]:
# Your Code
query6 = {"category_code": None}
cat_cod = (list(db.companies.find(query6, {"name":1})))
print(len(cat_cod))
#for element in cat_cod:
 #   print(f'Name of the company: {element["name"]}')

2751


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [128]:
# Your Code
query7 = {"$and":[{"number_of_employees": {"$gt":100}}, {"number_of_employees": {"$lt":1000}}]}
employees2 = (list(db.companies.find(query7, {"name":1, "number_of_employees":1})))
#for element in employees2:
 #   print(element["name"], element["number_of_employees"])
print(len(employees2))

753


### 9. Order all the companies by their IPO price in a descending order.

In [129]:
# Your Code
sorted_companies_ipo = list(db.companies.find().sort([("ipo",DESCENDING)]).limit(5))
for element in sorted_companies_ipo:
    print(element["name"])

GREE
Facebook
Amazon
Twitter
Groupon


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [130]:
# Your Code
ten_companies = list(db.companies.find().sort([("number_of_employees", DESCENDING)]).limit(10))
for element in ten_companies:
    print(element["name"])

Siemens
IBM
Toyota
PayPal
Nippon Telegraph and Telephone Corporation
Samsung Electronics
Accenture
Tata Consultancy Services
Flextronics International
Safeway


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [143]:
# Your Code
query8 = {"founded_month": {"$gte": 7}}
semester = (list(db.companies.find(query8, {"name":1})))
# can add .limit(1000)
#for element in semester:
#    print(element['name'])
print(len(semester))

3329


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [144]:
# Your Code
query9 = {"$and":[{"founded_year": {"$lt": 2000}}, {"acquisition.price_amount": {"$gt": 10000}}]}
comps = (list(db.companies.find(query9)))
#for element in comps:
#    print(element['name'])
print(len(comps))

225


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [154]:
# Your Code
query10 = {"acquisition.acquired_year": {"$gt": 2010}}
acq = (list(db.companies.find(query10, {"name":1, "acquisition":1})
                .sort([("acquisition.acquired_year",ASCENDING)])))
#for element in acq:
#    print(element['name'],element['acquisition'])
print(len(acq))

736


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [170]:
# Your Code

fourteen = (list(db.companies.find({},{"name":1, "founded_year":1}).sort([("founded_year",DESCENDING)]).limit(1000)))
#for element in fourteen:
#    print(element['name'],element['founded_year'])

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [177]:
# Your Code
query11 = {"founded_day": {"$lte": 7}}
days = (list(db.companies.find(query11).sort([("acquisition.price_amount",-1)]).limit(10)))
for element in days:
    print(element['name'])
print(len(days))

Netscape
PayPal
Zappos
Alibaba
Postini
Danger
Clearwell Systems
PrimeSense
Amobee
BlueLithium
10


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [179]:
# Your Code
query12 = {"$and":[{"category_code": "web"}, {"number_of_employees": {"$gt": 4000}}]}
web = (list(db.companies.find(query12).sort([("number_of_employees", ASCENDING)]).limit(10)))
for element in web:
    print(element['name'])
print(len(web))

Expedia
AOL
Webkinz
Rakuten
Los Angeles Times Media Group
Groupon
Yahoo!
eBay
Experian
9


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [183]:
# Your Code
query13 = {"$and":[{"acquisition.price_currency_code": "EUR"}, {"acquisition.price_amount": {"$gt": 10000000}}]}
eur = (list(db.companies.find(query13).limit(1000)))
for element in eur:
    print(element['name'])
print(len(eur))

ZYB
Apertio
Greenfield Online
Webedia
Wayfinder
Tuenti Technologies
BioMed Central
7


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [188]:
# Your Code
query14 = {"acquisition.acquired_month": {"$lte": 3}}
trimester = (list(db.companies.find(query14, {"name":1}).limit(10)))
for element in trimester:
    print(element['name'])
print(len(trimester))

Kyte
NetRatings
blogTV
Livestream
iContact
Coghead
Dailymotion
Netvibes
Flickr
BabyCenter
10


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code